In [1]:
import torch.nn.functional as F  # functions of the neural network library
import load_dataset as load  # module with function to load data
from Conv_Layer import *
from Pooling import *
import gc
from Dense import *

In [2]:
I = 28
O = I//2
n = 2*I
k = 2
K = I//4

device = torch.device("mps")
batch_size = 5  # the number of examples per batch
train_loader, test_loader, dim_in, dim_out = load.load_MNIST(batch_size=batch_size)
scala = 1000
reduced_loader = reduce_MNIST_dataset(test_loader, scala)
list_gates = [(i, j) for i in range(O-1) for j in range(i+1, O-1)]

In [3]:
full_model = nn.Sequential(Conv_RBS_density_I2(I,K,device), Pooling_2D_density(I,O,device),
                           Conv_RBS_density_I2(I//2,K,device), Pooling_2D_density(I//2,O//2,device),
                           Basis_Change_I_to_HW_density(O//2, device), Dense_RBS_density(O//2, list_gates, device))

In [4]:
loss_function = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(full_model.parameters(), lr=1)

def train_net(network, train_loader, loss_function, optimizer, device):
    network.train()
    train_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        # batch_data tensor size: batch_size * 784
        # init_density_matrix tensor size: batch_size * 784 * 784
        init_density_matrix = to_density_matrix(F.normalize(data.squeeze().resize(data.size()[0],784), p=2, dim=1).to(device), device)
        out_network = network(init_density_matrix) # out tensor size: batch * 91 * 91
        targets = get_batch_projectors(target, batch_size, int(binom(O,2)), device)
        loss = loss_function(out_network,targets)

        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        del out_network
        gc.collect()

    train_loss /= (batch_idx + 1)
    return train_loss

In [5]:
for i in range(5):
    train_loss = train_net(full_model, reduced_loader, loss_function, optimizer, device)
    print(train_loss)

/Users/letao/anaconda3/envs/env0/lib/python3.11/site-packages/torch/_tensor.py:885: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


44.236677169799805
44.13352394104004
44.05485534667969
43.986385345458984
43.859893798828125
